# US mobile air quality stations

OpenAQ API currently doesn't support filtering by mobile stations. Here's a quick way to see which ones are mobile. 

A mobile station's name stays the same as it moves around, but its coordinates change. A station's change in coordinates is treated as relocation. So technically it will be seen as a new station.

If you want to run these yourself, make sure you install necessary tools with `pipenv install` before starting the notebook.

In [4]:
import httpx
import itertools
from IPython.display import GeoJSON
import geojson

In [5]:
OPENAQADDR = "https://api.openaq.org/v1/locations"

In [6]:
response = httpx.get(OPENAQADDR, params={'country': 'US', 'limit': 10000})

In [7]:
stations = response.json()['results']

## Stations that changed their position more than 5 times

In [20]:
groupby_location = lambda station: station['location']

groups = [(group_name, list(group)) for (group_name, group) in itertools.groupby(stations, key=groupby_location)]
groups = sorted(groups, key=lambda g: len(g[1]), reverse=True)
groups = list(filter(lambda g: len(g[1]) > 5, groups))

for group_name, group in groups:
    print(group_name, len(group))
    


MMCA81033 205
GBUAPCD EBAM @ Lee V 126
GBUAPCD EBAM @ June 122
MMCA81010 120
MMFRA1001 115
Cle Elum_Mobile 101
MMCA81014 93
GBUAPCD EBAM @ Bridg 93
Unit153 92
GBUAPCD EBAM @ Alpin 85
GBUAPCD EBAM @ Bento 67
Plain_Mobile 65
GBUAPCD EBAM @ Crowl 62
MMCA81006 57
MMCA81036 54
MMCA81042 52
GBUAPCD EBAM @ Lone 50
Unit1019 46
Unit1029 45
MMCA81023 44
Unit0216 42
MMCA81015 38
Mobile_Newport 37
Unit0269 37
Leavenworth_Mobile 35
GBUAPCD EBAM @ Woodf 35
MMCA81030 33
Mobile_WhiteSalmon 32
MMCA81024 32
MMCA81051 32
MMCA81012 31
MMCA81032 29
Santa Rosa Fire Stat 28
MMNPS1017 28
Mobile_Cle Elum 25
MMCA81044 25
Vichy Elementary Sch 25
Mobile-Goldendale 24
Rincon Valley School 23
Unit386 22
MMCA81027 21
MMNPS1027 21
Hidden Valley School 20
GBUAPCD EBAM @ Coso 19
MMCA81016 18
MMCA81011 18
MMCA81008 17
MMCA81026 17
Unit1027 17
Unit328 16
Newport_Mobile 15
MMCA81009 15
MMCA81017 15
MMCA81039 15
MMCA81046 15
GBUAPCD EBAM 1 15
Brewster_Mobile 14
MMCA81001 11
GBUAPCD EBAM @ Bisho 10
MMCA81038 7
1001 6
1008 6

## Last known positions of these mobile stations

[See the result GeoJSON file: `us-mobile-stations.geojson`](us-mobile-stations.geojson)

Also would be cool to visualize the path they took.

In [47]:
from IPython.display import GeoJSON
import geojson
from datetime import datetime

def station_to_feature(station):
    omit_fields = ['cities', 'locations', 'sourceNames', 'sourceTypes']
    for field in omit_fields:
        if field in station:
            del station[field]
    return geojson.Feature(geometry=geojson.Point(tuple(station['coordinates'].values())), properties=station)

features = []

date_format = "%Y-%m-%dT%H:%M:%S.%fZ"
def get_last_updated_date(station):
    last_updated = station['lastUpdated']
    last_updated = datetime.strptime(last_updated, date_format)
    return last_updated

for _, group in groups:
    last_record = sorted(group, key=get_last_updated_date, reverse=True)[0]
    features.append(station_to_feature(last_record))
    
collection = geojson.FeatureCollection(features)
with open("us-mobile-stations.geojson", "w") as f:
    geojson.dump(collection, f)

GeoJSON(collection)

<IPython.display.GeoJSON object>